In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import ast
import typing

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet as wn

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import RobustScaler

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
from recettes_et_sentiments.api_model import rs_data, preprocessing, preprocessing_pipeline


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [47]:
recipes_p = pd.read_parquet("../batch-1672-recettes-et-sentiments-data/basic_preproc_recipes.parquet")
recipes = pd.read_csv('../batch-1672-recettes-et-sentiments-data/RAW_recipes.csv')
reviews = pd.read_csv('../batch-1672-recettes-et-sentiments-data/RAW_interactions.csv')


In [48]:
recipes.rename(columns={'id': 'recipe_id'}, inplace=True)


In [50]:
recipes_p.index.names = ['id']
recipes_p.head()

,name,minutes,contributor_id,submitted,tags,n_steps,description,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,merged_ingredients
id,,,,,,,,,,,,,,,,,
137739,arriba bake winter squash mexican style,0.0,47892,2005-09-16,minute less main north american side dish vege...,11,autumn favorite year cook prepare either spicy...,7,-0.758361,-0.606061,-0.203390,-0.500000,-0.363636,-0.511111,-0.416667,make choice proceed depend size squash cut fou...,winter squash mexican seasoning mixed spice ho...
31490,bit different breakfast pizza,0.0,26278,2002-06-17,minute less main north american breakfast main...,9,call crust prebaked bit add ingredient feel fr...,6,-0.405386,-0.060606,-0.423729,0.107143,0.090909,0.266667,-0.666667,preheat oven degrees press dough bottom side p...,prepared pizza crust sausage patty egg milk sa...
112140,kitchen chili,0.0,196586,2005-02-25,main chili crock pot slow cooker dietary hour ...,6,modify version mom chili hit christmas party m...,13,-0.126249,0.060606,0.118644,1.214286,0.477273,0.088889,-0.333333,brown ground beef large pot chop onion ground ...,ground beef yellow onion dice tomato tomato pa...
59389,alouette potato,0.0,68585,2003-04-14,minute less main side dish egg dairy potato ve...,11,super easy great taste make ahead side look li...,11,0.158390,-0.090909,-0.254237,-0.428571,-0.090909,-0.333333,0.916667,place potato large pot salt water gentle boil ...,spreadable cheese garlic herb new potato shall...
44061,amish tomato ketchup canning,0.0,41706,2002-10-25,weeknight main north american canning condimen...,5,dh amish mother raise much prefers store buy k...,8,0.114377,-0.575758,5.288136,0.321429,-0.340909,-0.511111,1.583333,mix ingredient boil hour thick jar old glass k...,tomato juice apple cider vinegar sugar salt pe...


In [49]:
ratings_rview_cnt = reviews.groupby("recipe_id")[['rating']].agg(
                                mean_rating=('rating', 'mean'),
                                review_count=('rating', 'count')
                                )


In [52]:
ratings_rview_cnt.index.names=['id']
ratings_rview_cnt.head()

,mean_rating,review_count
id,,
38,4.250000,4
39,3.000000,1
40,4.333333,9
41,4.500000,2
43,1.000000,1


In [54]:
merged = recipes_p.merge(ratings_rview_cnt, how="inner", on='id')
merged

,name,minutes,contributor_id,submitted,tags,n_steps,description,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,merged_ingredients,mean_rating,review_count
id,,,,,,,,,,,,,,,,,,,
137739,arriba bake winter squash mexican style,0.0,47892,2005-09-16,minute less main north american side dish vege...,11,autumn favorite year cook prepare either spicy...,7,-0.758361,-0.606061,-0.203390,-0.500000,-0.363636,-0.511111,-0.416667,make choice proceed depend size squash cut fou...,winter squash mexican seasoning mixed spice ho...,5.0,3
31490,bit different breakfast pizza,0.0,26278,2002-06-17,minute less main north american breakfast main...,9,call crust prebaked bit add ingredient feel fr...,6,-0.405386,-0.060606,-0.423729,0.107143,0.090909,0.266667,-0.666667,preheat oven degrees press dough bottom side p...,prepared pizza crust sausage patty egg milk sa...,3.5,4
112140,kitchen chili,0.0,196586,2005-02-25,main chili crock pot slow cooker dietary hour ...,6,modify version mom chili hit christmas party m...,13,-0.126249,0.060606,0.118644,1.214286,0.477273,0.088889,-0.333333,brown ground beef large pot chop onion ground ...,ground beef yellow onion dice tomato tomato pa...,4.0,1
59389,alouette potato,0.0,68585,2003-04-14,minute less main side dish egg dairy potato ve...,11,super easy great taste make ahead side look li...,11,0.158390,-0.090909,-0.254237,-0.428571,-0.090909,-0.333333,0.916667,place potato large pot salt water gentle boil ...,spreadable cheese garlic herb new potato shall...,4.5,2
44061,amish tomato ketchup canning,0.0,41706,2002-10-25,weeknight main north american canning condimen...,5,dh amish mother raise much prefers store buy k...,8,0.114377,-0.575758,5.288136,0.321429,-0.340909,-0.511111,1.583333,mix ingredient boil hour thick jar old glass k...,tomato juice apple cider vinegar sugar salt pe...,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486161,zydeco soup,0.0,227978,2012-08-29,ham minute less main north american soup stew ...,7,delicious soup originally find good home garde...,22,0.294773,0.181818,0.152542,0.428571,0.590909,-0.044444,0.500000,heat oil quart dutch oven celery onion sweet p...,celery onion green sweet pepper garlic clove o...,5.0,6
493372,zydeco spice mix,0.0,1500678,2013-01-09,minute less large group condiment etc easy veg...,1,spice mix make bud dance,13,-0.864630,-0.606061,-0.389831,1.571429,-0.386364,-0.511111,-0.666667,mix ingredient thoroughly,paprika salt garlic powder onion powder dry ba...,5.0,1
308080,zydeco ya ya devil egg,0.0,37779,2008-06-07,minute less main appetizer egg dairy easy fing...,7,devil egg cajun style,8,-0.736065,-0.424242,-0.389831,-0.392857,-0.272727,-0.400000,-0.750000,bowl mash yolk mayonnaise stir mustard stir ca...,hard egg mayonnaise dijon mustard salt free ca...,5.0,5


In [28]:
rs = RobustScaler()
merged[['minutes']] = rs.fit_transform(recipes[['minutes']])
merged.head()

,name,minutes,contributor_id,submitted,tags,n_steps,description,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,merged_ingredients,mean_rating,review_count
recipe_id,,,,,,,,,,,,,,,,,,,
137739,arriba bake winter squash mexican style,0.333333,47892.0,2005-09-16,minute less main north american side dish vege...,11.0,autumn favorite year cook prepare either spicy...,7.0,-0.758361,-0.606061,-0.203390,-0.500000,-0.363636,-0.511111,-0.416667,make choice proceed depend size squash cut fou...,winter squash mexican seasoning mixed spice ho...,5.0,3
31490,bit different breakfast pizza,-0.222222,26278.0,2002-06-17,minute less main north american breakfast main...,9.0,call crust prebaked bit add ingredient feel fr...,6.0,-0.405386,-0.060606,-0.423729,0.107143,0.090909,0.266667,-0.666667,preheat oven degrees press dough bottom side p...,prepared pizza crust sausage patty egg milk sa...,3.5,4
112140,kitchen chili,2.000000,196586.0,2005-02-25,main chili crock pot slow cooker dietary hour ...,6.0,modify version mom chili hit christmas party m...,13.0,-0.126249,0.060606,0.118644,1.214286,0.477273,0.088889,-0.333333,brown ground beef large pot chop onion ground ...,ground beef yellow onion dice tomato tomato pa...,4.0,1
59389,alouette potato,0.111111,68585.0,2003-04-14,minute less main side dish egg dairy potato ve...,11.0,super easy great taste make ahead side look li...,11.0,0.158390,-0.090909,-0.254237,-0.428571,-0.090909,-0.333333,0.916667,place potato large pot salt water gentle boil ...,spreadable cheese garlic herb new potato shall...,4.5,2
44061,amish tomato ketchup canning,3.333333,41706.0,2002-10-25,weeknight main north american canning condimen...,5.0,dh amish mother raise much prefers store buy k...,8.0,0.114377,-0.575758,5.288136,0.321429,-0.340909,-0.511111,1.583333,mix ingredient boil hour thick jar old glass k...,tomato juice apple cider vinegar sugar salt pe...,5.0,1


In [43]:
merged

,minutes,contributor_id,submitted,tags,n_steps,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,mean_rating,review_count,merged_text
recipe_id,,,,,,,,,,,,,,,,,
137739,0.333333,47892.0,2005-09-16,minute less main north american side dish vege...,11.0,7.0,-0.758361,-0.606061,-0.203390,-0.500000,-0.363636,-0.511111,-0.416667,make choice proceed depend size squash cut fou...,5.0,3,arriba bake winter squash mexican style\nautum...
31490,-0.222222,26278.0,2002-06-17,minute less main north american breakfast main...,9.0,6.0,-0.405386,-0.060606,-0.423729,0.107143,0.090909,0.266667,-0.666667,preheat oven degrees press dough bottom side p...,3.5,4,bit different breakfast pizza\ncall crust preb...
112140,2.000000,196586.0,2005-02-25,main chili crock pot slow cooker dietary hour ...,6.0,13.0,-0.126249,0.060606,0.118644,1.214286,0.477273,0.088889,-0.333333,brown ground beef large pot chop onion ground ...,4.0,1,kitchen chili\nmodify version mom chili hit ch...
59389,0.111111,68585.0,2003-04-14,minute less main side dish egg dairy potato ve...,11.0,11.0,0.158390,-0.090909,-0.254237,-0.428571,-0.090909,-0.333333,0.916667,place potato large pot salt water gentle boil ...,4.5,2,alouette potato\nsuper easy great taste make a...
44061,3.333333,41706.0,2002-10-25,weeknight main north american canning condimen...,5.0,8.0,0.114377,-0.575758,5.288136,0.321429,-0.340909,-0.511111,1.583333,mix ingredient boil hour thick jar old glass k...,5.0,1,amish tomato ketchup canning\ndh amish mother ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493372,0.444444,1500678.0,2013-01-09,minute less large group condiment etc easy veg...,1.0,13.0,-0.864630,-0.606061,-0.389831,1.571429,-0.386364,-0.511111,-0.666667,mix ingredient thoroughly,5.0,1,zydeco spice mix\nspice mix make bud dance\npa...
308080,-0.777778,37779.0,2008-06-07,minute less main appetizer egg dairy easy fing...,7.0,8.0,-0.736065,-0.424242,-0.389831,-0.392857,-0.272727,-0.400000,-0.750000,bowl mash yolk mayonnaise stir mustard stir ca...,5.0,5,zydeco ya ya devil egg\ndevil egg cajun style\...
298512,0.000000,506822.0,2008-04-15,minute less large group hand form cooky desser...,9.0,10.0,-0.363110,-0.272727,0.542373,-0.107143,-0.250000,-0.044444,0.000000,place melt butter large mix bowl list beat wel...,1.0,1,cooky design cooky stick\nheard cooky design c...


In [46]:
recipes.iloc[44061]

name                              chicken enchilada pasta enchilada
recipe_id                                                    264678
minutes                                                          35
contributor_id                                               629515
submitted                                                2007-11-10
tags              ['60-minutes-or-less', 'time-to-make', 'course...
nutrition            [1019.2, 43.0, 21.0, 106.0, 102.0, 54.0, 46.0]
n_steps                                                          28
steps             ['pasta:', 'prepare rigatoni pasta per package...
description       to prepare this dish get everything prepped an...
ingredients       ['rigatoni pasta', 'salt', 'corn oil', 'skinle...
n_ingredients                                                    23
Name: 44061, dtype: object

In [55]:
merged.to_parquet("../batch-1672-recettes-et-sentiments-data/basic_preproc_recipes_with_reviews_score.parquet")

In [38]:
def concat_columns(df:pd.DataFrame, columns:typing.List[str], dropSourceColumn:bool=True)->None:

    df['merged_text'] = df[columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


    if dropSourceColumn:
        df.drop(columns=columns,axis=1, inplace=True)

    return df


In [39]:

concat_columns(merged, ["name", "description", "merged_ingredients"])

,minutes,contributor_id,submitted,tags,n_steps,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,mean_rating,review_count,merged_text
recipe_id,,,,,,,,,,,,,,,,,
137739,0.333333,47892.0,2005-09-16,minute less main north american side dish vege...,11.0,7.0,-0.758361,-0.606061,-0.203390,-0.500000,-0.363636,-0.511111,-0.416667,make choice proceed depend size squash cut fou...,5.0,3,arriba bake winter squash mexican style\nautum...
31490,-0.222222,26278.0,2002-06-17,minute less main north american breakfast main...,9.0,6.0,-0.405386,-0.060606,-0.423729,0.107143,0.090909,0.266667,-0.666667,preheat oven degrees press dough bottom side p...,3.5,4,bit different breakfast pizza\ncall crust preb...
112140,2.000000,196586.0,2005-02-25,main chili crock pot slow cooker dietary hour ...,6.0,13.0,-0.126249,0.060606,0.118644,1.214286,0.477273,0.088889,-0.333333,brown ground beef large pot chop onion ground ...,4.0,1,kitchen chili\nmodify version mom chili hit ch...
59389,0.111111,68585.0,2003-04-14,minute less main side dish egg dairy potato ve...,11.0,11.0,0.158390,-0.090909,-0.254237,-0.428571,-0.090909,-0.333333,0.916667,place potato large pot salt water gentle boil ...,4.5,2,alouette potato\nsuper easy great taste make a...
44061,3.333333,41706.0,2002-10-25,weeknight main north american canning condimen...,5.0,8.0,0.114377,-0.575758,5.288136,0.321429,-0.340909,-0.511111,1.583333,mix ingredient boil hour thick jar old glass k...,5.0,1,amish tomato ketchup canning\ndh amish mother ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493372,0.444444,1500678.0,2013-01-09,minute less large group condiment etc easy veg...,1.0,13.0,-0.864630,-0.606061,-0.389831,1.571429,-0.386364,-0.511111,-0.666667,mix ingredient thoroughly,5.0,1,zydeco spice mix\nspice mix make bud dance\npa...
308080,-0.777778,37779.0,2008-06-07,minute less main appetizer egg dairy easy fing...,7.0,8.0,-0.736065,-0.424242,-0.389831,-0.392857,-0.272727,-0.400000,-0.750000,bowl mash yolk mayonnaise stir mustard stir ca...,5.0,5,zydeco ya ya devil egg\ndevil egg cajun style\...
298512,0.000000,506822.0,2008-04-15,minute less large group hand form cooky desser...,9.0,10.0,-0.363110,-0.272727,0.542373,-0.107143,-0.250000,-0.044444,0.000000,place melt butter large mix bowl list beat wel...,1.0,1,cooky design cooky stick\nheard cooky design c...


In [30]:
reviews.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [32]:
reviews.groupby('user_id')[['rating']].agg(
                                mean_rating=('rating', 'mean'),
                                review_count=('rating', 'count')
                                ).describe()

,mean_rating,review_count
count,226570.000000,226570.000000
mean,3.873031,4.997868
std,1.773173,49.663111
min,0.000000,1.000000
25%,3.800000,1.000000
50%,5.000000,1.000000
75%,5.000000,2.000000
max,5.000000,7671.000000


In [33]:
reviews.groupby('user_id')[['rating']].agg(
                                mean_rating=('rating', 'mean'),
                                review_count=('rating', 'count')
                                ).sort_values('review_count')

,mean_rating,review_count
user_id,,
1579230,4.000000,1
2744132,4.000000,1
2744161,5.000000,1
2744177,0.000000,1
2744231,5.000000,1
...,...,...
128473,4.908348,3917
169430,4.693327,4076
383346,4.755186,4628


# Trash code

In [27]:
def get_stopwords(type:int=0)->set:
    """
    Return english (stopwords) from NLTK if type=0

    TODO : support stopwords customized for our use case by passing other value for type

    Args:
        type (int): Type of stopwords to return.
            0 => default english NLTK stopwords
            Other value raise ValueError as of now

    Returns:
        Set[str]: set of stopwords

    Raises:
        ValueError: if type is not supported
    """
    stop_words = set(stopwords.words('english'))

    if type == 0:
        return stop_words

    raise ValueError(f"type {type} not supported")



def remove_stopwords(text:str, stop_words:set)->str:
    """
    Return the "text" argument without the stop_words


    Args:
        text (str): text to be processed.
        stop_words (set) : set of stop words to be used for processing the text.

    Returns:
        str: the input text without stop words
    """
    tokens = word_tokenize(text)
    tokenized_sentence_no_stopwords = [w for w in tokens if not w in stop_words]
    return ' '.join(tokenized_sentence_no_stopwords)


def lemmatize_text(text:str)->str:
    """
    Return the lemmatize "text"

    Args:
        text (str): text to be processed. Text should have been preprocessed (lowercase, ponctuation removed, stopwords removed)

    Returns:
        str: the input text without stop words
    """
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wn.ADJ
        elif treebank_tag.startswith('V'):
            return wn.VERB
        elif treebank_tag.startswith('N'):
            return wn.NOUN
        elif treebank_tag.startswith('R'):
            return wn.ADV
        else:
            return wn.NOUN  # Par défaut, retourne NOM

    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

    lemmatizer = WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in pos_tags]

    return ' '.join(lemmatized_tokens)

In [11]:
df.iloc[0]['merged_steps']

'make choice proceed recipe depending size squash cut half fourth remove seed spicy squash drizzle olive oil melted butter cut squash piece season mexican seasoning mix ii sweet squash drizzle melted honey butter grated piloncillo cut squash piece season sweet mexican spice mix bake degree depending size minute hour fork easily pierce skin careful burn squash especially opt use sugar butter feel comfortable cover squash aluminum foil first half hour give take baking desired season salt'

In [28]:
stop_words = get_stopwords()

In [36]:
df['merged_ingredients'] = df['ingredients'].apply(lambda x: "\n".join(x))


In [38]:
df['merged_steps'] = df['steps'].apply(lambda x: "\n".join(x))
df['merged_tags'] = df['tags'].apply(lambda x: "\n".join(x))

In [39]:
df['merged_ingredients_wo_sw'] = df['merged_ingredients'].apply(lambda x: remove_stopwords(x, stop_words=stop_words))

In [40]:
df['merged_ingredients_wo_sw_lem'] = df['merged_ingredients_wo_sw'].apply(lemmatize_text)


In [19]:
def count_vectorize(
    df:pd.DataFrame,
    column_name:str,
    min_df:float=1.0,
    max_df:float=1.0,
    max_features:int=None,
    ngram_range=(1,1)
    )->pd.DataFrame:
    """
    usage :
    CountVectorize the text in column column_name and add to the Data Frame the new columns

    Args:
        df (pd.DataFrame): DataFrame in which there is a preprocessed text column to be vectorized
        colum_name (str) : name of the column that contains the text to CountVectorize

    Returns:
        pd.DataFrame

    """

    count_vectorizer = CountVectorizer(
        min_df=min_df,
        max_df=max_df,
        max_features=max_features,
        ngram_range=ngram_range
        )

    X = count_vectorizer.fit_transform(df[column_name])
    X_df = pd.DataFrame(X.toarray(), columns = count_vectorizer.get_feature_names_out())

    return pd.concat([df, X_df], axis = 1)


In [13]:

df2 = preprocessing.count_vectorize(
    df,
    'merged_steps',
    min_df=0.1,
    max_df=0.95
    )
df2

,name,minutes,contributor_id,submitted,tags,n_steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,add,apple,aside,bake,baking,beat,boil,bottom,bowl,bring,brown,butter,cheese,cinnamon,combine,cook,cooking,cool,cover,cream,cup,cut,degree,dish,drain,dry,egg,flour,garlic,golden,half,heat,high,...,low,make,medium,milk,minute,mix,mixture,oil,one,onion,oven,pan,pepper,place,pour,powder,preheat,put,rack,remaining,remove,salt,sauce,saucepan,serve,serving,set,sheet,side,simmer,skillet,slice,small,smooth,spoon,spread,sprinkle,stir,stirring,sugar,tender,time,together,top,turn,using,vanilla,warm,water,well
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,minute le course main ingredient cuisine prepa...,11,autumn favorite time year cook recipe prepared...,make choice proceed recipe depending size squa...,7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,make choice proceed recipe depending size squa...,0,0,0,1,1,0,0,0,0,0,0,3,0,0,0,0,0,0,1,0,0,3,1,0,0,0,0,0,0,0,2,0,0,...,0,1,0,0,1,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
31490,bit different breakfast pizza,30,26278,2002-06-17,minute le course main ingredient cuisine prepa...,9,recipe call crust prebaked bit adding ingredie...,preheat oven degree f press dough bottom side ...,6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,preheat oven degree f press dough bottom side ...,0,0,0,2,0,0,0,1,1,0,1,0,2,0,0,0,0,0,0,0,0,1,1,0,0,0,3,0,0,0,0,0,0,...,0,0,0,1,2,0,1,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
112140,kitchen chili,130,196586,2005-02-25,course preparation main dish chili crock pot s...,6,modified version mom chili hit christmas party...,brown ground beef large pot add chopped onion ...,13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,brown ground beef large pot add chopped onion ...,3,0,0,0,0,0,0,0,0,0,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59389,alouette potato,45,68585,2003-04-14,minute le course main ingredient preparation o...,11,super easy great tasting make ahead side dish ...,place potato large pot lightly salted water br...,11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,place potato large pot lightly salted water br...,1,0,1,1,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0,1,0,1,2,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1
44061,amish tomato ketchup canning,190,41706,2002-10-25,weeknight course main ingredient cuisine prepa...,5,dh amish mother raised recipe much prefers sto...,mix ingredient boil hour thick pour jar use ol...,8,352.9,1.0,337.0,23.0,3.0,0.0,28.0,mix ingredient boil hour thick pour jar use ol...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81712,apricot barbecued chicken,45,67899,2004-01-21,minute le course main ingredient preparation o...,6,look scrumptious sweet wait try jan feb issue ...,large skillet cook chicken garlic oil browned ...,8,266.6,7.0,61.0,9.0,53.0,4.0,9.0,large skillet cook chicken garlic oil browned ...,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,2,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
107482,apricot barley cass

In [10]:



def tfidf_vectorize(
    df:pd.DataFrame,
    column_name:str,
    min_df:float=1.0,
    max_df:float=1.0,
    max_features:int=None,
    ngram_range=(1,1))->pd.DataFrame:
    """
    usage :
    TFIDF Vectorize the text in column column_name and add to the Data Frame the new columns

    Args:
        df (pd.DataFrame): DataFrame in which there is a preprocessed text column to be vectorized
        colum_name (str) : name of the column that contains the text to Vectorize

    Returns:
        pd.DataFrame

    """
    tf_idf_vectorizer = TfidfVectorizer(
        min_df=min_df,
        max_df=max_df,
        max_features=max_features,
        ngram_range=ngram_range
        )

    X = tf_idf_vectorizer.fit_transform(df[column_name])
    X_df = pd.DataFrame(X.toarray(), columns = tf_idf_vectorizer.get_feature_names_out())

    return pd.concat([df, X_df], axis = 1)

In [16]:
df3 = preprocessing.tfidf_vectorize(
    df,
    'merged_steps',
    min_df=0.1,
    max_df=0.95)
df3

,name,minutes,contributor_id,submitted,tags,n_steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,add,apple,aside,bake,baking,beat,boil,bottom,bowl,bring,brown,butter,cheese,cinnamon,combine,cook,cooking,cool,cover,cream,cup,cut,degree,dish,drain,dry,egg,flour,garlic,golden,half,heat,high,...,low,make,medium,milk,minute,mix,mixture,oil,one,onion,oven,pan,pepper,place,pour,powder,preheat,put,rack,remaining,remove,salt,sauce,saucepan,serve,serving,set,sheet,side,simmer,skillet,slice,small,smooth,spoon,spread,sprinkle,stir,stirring,sugar,tender,time,together,top,turn,using,vanilla,warm,water,well
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,minute le course main ingredient cuisine prepa...,11,autumn favorite time year cook recipe prepared...,make choice proceed recipe depending size squa...,7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,make choice proceed recipe depending size squa...,0.000000,0.0,0.000000,0.122859,0.153878,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.419597,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.159984,0.0,0.0,0.525881,0.168692,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.39574,0.000000,0.000000,...,0.000000,0.195715,0.000000,0.00000,0.091380,0.242439,0.000000,0.159254,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.152523,0.132793,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.132334,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
31490,bit different breakfast pizza,30,26278,2002-06-17,minute le course main ingredient cuisine prepa...,9,recipe call crust prebaked bit adding ingredie...,preheat oven degree f press dough bottom side ...,6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,preheat oven degree f press dough bottom side ...,0.000000,0.0,0.000000,0.247613,0.000000,0.000000,0.000000,0.211945,0.123508,0.000000,0.160238,0.000000,0.350030,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.176645,0.169992,0.000000,0.000000,0.0,0.463946,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.18425,0.184168,0.000000,0.130425,0.000000,0.0,0.000000,0.135129,0.136372,0.000000,0.000000,0.146757,0.000000,0.163403,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.346341,0.0,0.183553,0.000000,0.000000,0.0,0.183701,0.0,0.204104,0.000000,0.163331,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
112140,kitchen chili,130,196586,2005-02-25,course preparation main dish chili crock pot s...,6,modified version mom chili hit christmas party...,brown ground beef large pot add chopped onion ...,13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,brown ground beef large pot add chopped onion ...,0.411224,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.444370,0.000000,0.242676,0.0,0.000000,0.204756,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.283104,...,0.268607,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.236440,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.19486,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
59389,alouette potato,45,68585,2003-04-14,minute le course main ingredient preparation o...,11,super easy great tasting make ahead side dish ...,place potato large pot lightly salted water br...,11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,place potato large pot lightly salted water br...,0.099938,0.0,0.19

In [13]:
my_data.iloc[231635]

name                         cookies by design   cookies on a stick
id                                                           298512
minutes                                                          29
contributor_id                                               506822
submitted                                       2008-04-15 00:00:00
tags              ['30-minutes-or-less', 'time-to-make', 'course...
nutrition                 [188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]
n_steps                                                           9
steps             ['place melted butter in a large mixing bowl a...
description       i've heard of the 'cookies by design' company,...
ingredients       ['butter', 'eagle brand condensed milk', 'ligh...
n_ingredients                                                    10
Name: 231635, dtype: object

In [57]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
class ConcatColumns(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['merged_text'] = X[self.columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        if self.dropSourceColumn:
            X.drop(columns=self.columns, axis=1, inplace=True)
        return X

In [75]:
from recettes_et_sentiments.api_model import preprocessing_pipeline

preprocessor_pipeline = preprocessing_pipeline.make_preprocessor_pipeline(
    use_count_vectorizer=True,
    columns_to_merge_for_training=["name", "description", "merged_ingredients"],
    )


In [68]:
preprocessor_pipeline

Pipeline(steps=[('basic_preproc', BasicPreprocessing()),
                ('concat_columns',
                 ConcatColumns(columns=['name', 'description',
                                        'merged_ingredients'])),
                ('vectorize_and_combine',
                 ColumnTransformer(transformers=[('text',
                                                  ('CountVectorizer',
                                                   CountVectorizer(max_df=0.95,
                                                                   min_df=0.1)),
                                                  'merged_text')]))])

In [4]:
recipe_df_ori = rs_data.load_recipes("../batch-1672-recettes-et-sentiments-data/RAW_recipes.csv")


2024-06-06 11:02:50 - recettes_et_sentiments.api_model.rs_data - INFO - loading '../batch-1672-recettes-et-sentiments-data/RAW_recipes.csv'


2024-06-06 11:02:55 - recettes_et_sentiments.api_model.rs_data - INFO - loading '../batch-1672-recettes-et-sentiments-data/RAW_recipes.csv' done.
2024-06-06 11:02:55 - recettes_et_sentiments.api_model.rs_data - INFO - Starting add_columns_and_merge_text processing
2024-06-06 11:03:24 - recettes_et_sentiments.api_model.rs_data - INFO - add_columns_and_merge_text done


In [11]:
min_df=6
max_df=0.98
max_features=None
ngram_range=(1,1)
countVectorizer=False
file_suffix = f"{'CountVectorizer' if countVectorizer else 'TfIdfVectorizer'}_{min_df}_{max_df}_{max_features}_{str(ngram_range[0])+'_'+str(ngram_range[1])}"

preprocessor_pipeline = preprocessing_pipeline.make_preprocessor_pipeline(
    use_count_vectorizer=True,
    columns_to_merge_for_training=["name", "description", "merged_ingredients"],
    min_df=min_df,
    max_df=max_df,
    max_features=max_features,
    ngram_range=ngram_range,
    cache=True
)

recipe_processed = preprocessor_pipeline.fit_transform(recipe_df_ori)

recipe_processed.to_parquet(f"../batch-1672-recettes-et-sentiments-data/preproc_recipes_{file_suffix}.parquet")


2024-06-06 11:11:08 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - data/cache/basic_preproc_6_0.98_None_1_1.parquet does not exist. Skipping load.
2024-06-06 11:11:08 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - data/cache/concat_columns_name_description_merged_ingredients_6_0.98_None_1_1.parquet does not exist. Skipping load.
2024-06-06 11:11:08 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - FIT : No cached data available for data/cache/basic_preproc_6_0.98_None_1_1.parquet, fitting step
2024-06-06 11:11:08 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - Fitting BasicPreprocessing
2024-06-06 11:11:08 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - Transforming data and saving to data/cache/basic_preproc_6_0.98_None_1_1.parquet
2024-06-06 11:11:08 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - Transforming data with BasicPreprocessing


2024-06-06 11:42:23 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - Saving DataFrame to data/cache/basic_preproc_6_0.98_None_1_1.parquet
2024-06-06 11:42:24 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - FIT : No cached data available for data/cache/concat_columns_name_description_merged_ingredients_6_0.98_None_1_1.parquet, fitting step
2024-06-06 11:42:24 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - Fitting ConcatColumns
2024-06-06 11:42:24 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - Transforming data and saving to data/cache/concat_columns_name_description_merged_ingredients_6_0.98_None_1_1.parquet
2024-06-06 11:42:24 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - Transforming data with ConcatColumns
2024-06-06 11:42:27 - recettes_et_sentiments.api_model.preprocessing_pipeline - INFO - Saving DataFrame to data/cache/concat_columns_name_description_merged_ingredients_6_0.98_None_1_1

In [4]:
recipe_processed.to_parquet(f"../batch-1672-recettes-et-sentiments-data/preproc_recipes_tfidf.parquet")

AttributeError: module 'pandas' has no attribute 'to_parquet'

In [5]:
recipe_processed = pd.read_parquet(f"../batch-1672-recettes-et-sentiments-data/preproc_recipes_tfidfvectorizer_defaults.parquet")

In [6]:
reviews = pd.read_csv('../batch-1672-recettes-et-sentiments-data/RAW_interactions.csv')
preproc_with_reviews = rs_data.get_y(recipe_processed, reviews)
preproc_with_reviews.drop(columns=['remainder__merged_steps','remainder__tags','remainder__submitted'], axis=1, inplace=True)

2024-06-06 14:16:54 - recettes_et_sentiments.api_model.rs_data - INFO - Starting get_y processing
2024-06-06 14:17:41 - recettes_et_sentiments.api_model.rs_data - INFO - get_y done


In [5]:
for col in preproc_with_reviews.columns:
    if col.startswith("remainder__"):
        print (col+"\n")

remainder__minutes

remainder__contributor_id

remainder__n_steps

remainder__n_ingredients

remainder__calories

remainder__total_fat_pdv

remainder__sugar_pdv

remainder__sodium_pdv

remainder__protein_pdv

remainder__saturated_fat_pdv

remainder__carbohydrates_pdv



In [7]:
preproc_with_reviews.to_parquet(f"../batch-1672-recettes-et-sentiments-data/preproc_recipes_tfidfvectorizer_defaults_with_y.parquet")

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
import pandas as pd


In [2]:
preproc_with_reviews = pd.read_parquet(f"../batch-1672-recettes-et-sentiments-data/preproc_recipes_tfidfvectorizer_defaults_with_y.parquet")

In [4]:
preproc_with_reviews.drop(columns='remainder__merged_steps', axis=1, inplace=True)
preproc_with_reviews.head()


,text__aa,text__aaa,text__aah,text__aahs,text__aaron,text__aarp,text__aarti,text__aash,text__ab,text__abandon,...,remainder__n_ingredients,remainder__calories,remainder__total_fat_pdv,remainder__sugar_pdv,remainder__sodium_pdv,remainder__protein_pdv,remainder__saturated_fat_pdv,remainder__carbohydrates_pdv,mean_rating,count_rating
id,,,,,,,,,,,,,,,,,,,,,
137739,0,0,0,0,0,0,0,0,0,0,...,7,-0.758361,-0.606061,-0.203390,-0.500000,-0.363636,-0.511111,-0.416667,5.0,3
31490,0,0,0,0,0,0,0,0,0,0,...,6,-0.405386,-0.060606,-0.423729,0.107143,0.090909,0.266667,-0.666667,3.5,4
112140,0,0,0,0,0,0,0,0,0,0,...,13,-0.126249,0.060606,0.118644,1.214286,0.477273,0.088889,-0.333333,4.0,1
59389,0,0,0,0,0,0,0,0,0,0,...,11,0.158390,-0.090909,-0.254237,-0.428571,-0.090909,-0.333333,0.916667,4.5,2
44061,0,0,0,0,0,0,0,0,0,0,...,8,0.114377,-0.575758,5.288136,0.321429,-0.340909,-0.511111,1.583333,5.0,1


In [6]:
# preproc_with_reviews.drop(columns=['remainder__merged_steps','remainder__tags','remainder__submitted'], axis=1, inplace=True)

cv_nb = cross_validate(
    LinearRegression(n_jobs=-1),
    preproc_with_reviews.drop(columns='mean_rating'),
    preproc_with_reviews['mean_rating'],
    cv=5,
    n_jobs=-1,
    scoring='r2',
    verbose=2
)

#round(cv_nb['test_score'].mean(),2)
cv_nb

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


tf idf
et predire une note.

In [109]:
recipe_processed.head()

,text__aa,text__aaa,text__aah,text__aahs,text__aaron,text__aarp,text__aarti,text__aash,text__ab,text__abandon,text__abbey,text__abby,text__abc,text__abide,text__abigail,text__abilene,text__ability,text__abit,text__able,text__abm,text__abn,text__aboard,text__aboriginal,text__aborio,text__abound,text__abraham,text__abroad,text__absence,text__absinthe,text__absolut,text__absolute,text__absolutely,text__absolutley,text__absolutly,text__absorb,text__absorbed,text__absorbs,text__absorption,text__absoulutely,text__absurdly,text__abuelita,text__abundance,text__abundant,text__abuse,text__ac,text__academy,text__acadia,text__acadian,text__acapulco,text__accent,...,text__zinger,text__zingy,text__zip,text__ziploc,text__ziplock,text__ziplocks,text__ziplocs,text__zipper,text__zippy,text__ziti,text__zmail,text__zoe,text__zojirushi,text__zombie,text__zone,text__zoom,text__zsp,text__zucca,text__zucchine,text__zucchini,text__zucchinis,text__zuchini,text__zuchinni,text__zuke,text__zukes,text__zuni,text__zuppa,text__zurich,text__zurie,text__zw,text__zwieback,text__zwt,text__zwtii,text__zydeco,text__ºf,text__½s,remainder__minutes,remainder__contributor_id,remainder__submitted,remainder__tags,remainder__n_steps,remainder__n_ingredients,remainder__calories,remainder__total_fat_pdv,remainder__sugar_pdv,remainder__sodium_pdv,remainder__protein_pdv,remainder__saturated_fat_pdv,remainder__carbohydrates_pdv,remainder__merged_steps
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
137739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.333333,47892,2005-09-16,main north american side dish vegetable mexica...,11,7,-0.758361,-0.606061,-0.20339,-0.5,-0.363636,-0.511111,-0.416667,make choice proceed depend size squash cut fou...
31490,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.222222,26278,2002-06-17,main north american breakfast main pork americ...,9,6,-0.405386,-0.060606,-0.423729,0.107143,0.090909,0.266667,-0.666667,preheat oven degrees press dough bottom side p...
112140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,196586,2005-02-25,main chili crock pot slow cooker dietary hour ...,6,13,-0.126249,0.060606,0.118644,1.214286,0.477273,0.088889,-0.333333,brown ground beef large pot chop onion ground ...
59389,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.111111,68585,2003-04-14,main side dish egg dairy potato vegetable oven...,11,11,0.15839,-0.090909,-0.254237,-0.428571,-0.090909,-0.333333,0.916667,place potato large pot salt water gentle boil ...
44061,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,41706,2002-10-25,weeknight main north american canning condimen...,5,8,0.114377,-0.575758,5.288136,0.321429,-0.340909,-0.511111,1.583333,mix ingredient boil hour thick jar old glass k...


In [86]:
!pip list | grep learn

scikit-learn                 1.5.0


In [83]:
import sklearn
print(sklearn.__version__)

1.5.0


In [3]:
preproc = pd.read_parquet("data/cache/concat_columns_name_description_merged_ingredients.parquet")
preproc.head()

,minutes,contributor_id,submitted,tags,n_steps,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,merged_text
0,0.333333,47892,2005-09-16,main north american side dish vegetable mexica...,11,7,-0.758361,-0.606061,-0.203390,-0.500000,-0.363636,-0.511111,-0.416667,make choice proceed depend size squash cut fou...,arriba bake winter squash mexican style autumn...
1,-0.222222,26278,2002-06-17,main north american breakfast main pork americ...,9,6,-0.405386,-0.060606,-0.423729,0.107143,0.090909,0.266667,-0.666667,preheat oven degrees press dough bottom side p...,bit different breakfast pizza call crust preba...
2,2.000000,196586,2005-02-25,main chili crock pot slow cooker dietary hour ...,6,13,-0.126249,0.060606,0.118644,1.214286,0.477273,0.088889,-0.333333,brown ground beef large pot chop onion ground ...,kitchen chili modify version mom chili hit chr...
3,0.111111,68585,2003-04-14,main side dish egg dairy potato vegetable oven...,11,11,0.158390,-0.090909,-0.254237,-0.428571,-0.090909,-0.333333,0.916667,place potato large pot salt water gentle boil ...,alouette potato super easy great taste make ah...
4,2.000000,41706,2002-10-25,weeknight main north american canning condimen...,5,8,0.114377,-0.575758,5.288136,0.321429,-0.340909,-0.511111,1.583333,mix ingredient boil hour thick jar old glass k...,amish tomato ketchup canning dh amish mother r...


In [2]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [4]:
preproc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231636 entries, 0 to 231635
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   minutes            231636 non-null  float64       
 1   contributor_id     231636 non-null  int64         
 2   submitted          231636 non-null  datetime64[ns]
 3   tags               231636 non-null  object        
 4   n_steps            231636 non-null  int64         
 5   n_ingredients      231636 non-null  int64         
 6   calories           231636 non-null  float64       
 7   total_fat_pdv      231636 non-null  float64       
 8   sugar_pdv          231636 non-null  float64       
 9   sodium_pdv         231636 non-null  float64       
 10  protein_pdv        231636 non-null  float64       
 11  saturated_fat_pdv  231636 non-null  float64       
 12  carbohydrates_pdv  231636 non-null  float64       
 13  merged_steps       231636 non-null  object  

In [6]:
reviews = pd.read_csv('../batch-1672-recettes-et-sentiments-data/RAW_interactions.csv')
preproc_with_rating = rs_data.get_y(preproc,reviews)

2024-06-07 13:45:30 - recettes_et_sentiments.api_model.rs_data - INFO - Starting get_y processing


KeyError: 'id'

In [2]:
import pandas as pd
pd.read_parquet('../batch-1672-recettes-et-sentiments-data/last_preproc_we_hope.parquet')

,name,minutes,contributor_id,submitted,tags,n_steps,description,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,merged_ingredients,mean_rating,count_rating
id,,,,,,,,,,,,,,,,,,,
137739,arriba bake winter squash mexican style,0.333333,47892,2005-09-16,"b'[""occasion"",""north-american"",""side-dishes"",""...",11,autumn favorite year cook prepare either spicy...,7,-0.758361,-0.606061,-0.203390,-0.500000,-0.363636,-0.511111,-0.416667,make choice proceed depend size squash cut fou...,winter squash mexican seasoning mixed spice ho...,5.0,3
31490,bit different breakfast pizza,-0.222222,26278,2002-06-17,"b'[""occasion"",""north-american"",""breakfast"",""ma...",9,call crust prebaked bit add ingredient feel fr...,6,-0.405386,-0.060606,-0.423729,0.107143,0.090909,0.266667,-0.666667,preheat oven degrees press dough bottom side p...,prepared pizza crust sausage patty egg milk sa...,3.5,4
112140,kitchen chili,2.000000,196586,2005-02-25,"b'[""main-dish"",""chili"",""crock-pot-slow-cooker""...",6,modify version mom chili hit christmas party m...,13,-0.126249,0.060606,0.118644,1.214286,0.477273,0.088889,-0.333333,brown ground beef large pot chop onion ground ...,ground beef yellow onion dice tomato tomato pa...,4.0,1
59389,alouette potato,0.111111,68585,2003-04-14,"b'[""occasion"",""side-dishes"",""eggs-dairy"",""pota...",11,super easy great taste make ahead side look li...,11,0.158390,-0.090909,-0.254237,-0.428571,-0.090909,-0.333333,0.916667,place potato large pot salt water gentle boil ...,spreadable cheese garlic herb new potato shall...,4.5,2
44061,amish tomato ketchup canning,2.000000,41706,2002-10-25,"b'[""weeknight"",""occasion"",""north-american"",""ca...",5,dh amish mother raise much prefers store buy k...,8,0.114377,-0.575758,5.288136,0.321429,-0.340909,-0.511111,1.583333,mix ingredient boil hour thick jar old glass k...,tomato juice apple cider vinegar sugar salt pe...,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486161,zydeco soup,0.444444,227978,2012-08-29,"b'[""ham"",""north-american"",""soups-stews"",""pork""...",7,delicious soup originally find good home garde...,22,0.294773,0.181818,0.152542,0.428571,0.590909,-0.044444,0.500000,heat oil quart dutch oven celery onion sweet p...,celery onion green sweet pepper garlic clove o...,5.0,6
493372,zydeco spice mix,-0.777778,1500678,2013-01-09,"b'[""for-large-groups"",""condiments-etc"",""easy"",...",1,spice mix make bud dance,13,-0.864630,-0.606061,-0.389831,1.571429,-0.386364,-0.511111,-0.666667,mix ingredient thoroughly,paprika salt garlic powder onion powder dry ba...,5.0,1
308080,zydeco ya ya devil egg,0.000000,37779,2008-06-07,"b'[""appetizers"",""eggs-dairy"",""easy"",""finger-fo...",7,devil egg cajun style,8,-0.736065,-0.424242,-0.389831,-0.392857,-0.272727,-0.400000,-0.750000,bowl mash yolk mayonnaise stir mustard stir ca...,hard egg mayonnaise dijon mustard salt free ca...,5.0,5
